 Trying to pseudobulk the single-cell transcriptomic data: (E-MTAB-9357) <br>
More information on pseudo-bulking can be found here: <br>
https://decoupler-py.readthedocs.io/en/latest/notebooks/pseudobulk.html

The individual samples are accessible as individual txt files. <br>
More information on concatenating individual Scanpy objects can be found here: <br>
https://anndata.readthedocs.io/en/latest/generated/anndata.concat.html

More information on how to annotate metadata can be found here:<br>
https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html


Seems as though individual samples have already been pre-processed since the counts are not integers <br>
The authors seem to only have one layer of the normalised counts, and did not keep a raw count layer since the file is saved as a txt file and not as an AnnData object

Pre-processing steps are detailed in the Su 2020 paper and include:  <br>
-Filtering out cells expressing < 200 genes and >2,500 genes, that have UMI counts per cell >10,000, and with too many mitochondrial genes (proportion >10%) <br>
-Removal of doublets   <br>
-Normalisation (UMI total count set to 10^6 reads per cell) <br>
-Log transformation (natural log of CPM + 1) <br>
More information of pre-processing in Scanpy found here:  <br>
https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

Due to the number of samples, the analysis may have to be split into subsets <br>
Batch correction is used for visualisation purposes ONLY 

In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import decoupler as dc
import anndata
import os 

In [9]:
df = pd.read_csv("Data/scrnaseq/heathlab_dc_9_17_pbmc_gex_library_3_1.txt",index_col=0,delim_whitespace=True)
df

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
AAACCTGAGTGGGCTA-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTCGAGTTT-1-2:S5:3-1,7.280958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.280958,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTGTCGCTG-1-2:S5:3-1,6.016969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTTATCACG-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGTCAAGGTAA-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.287707,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGGATTCGG-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
TTTGTCAGTCAAAGAT-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.921827,0.0
TTTGTCAGTCATGCAT-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.085595,0.0
TTTGTCAGTTGATTGC-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.002662,0.0


In [12]:
transcript_df = pd.DataFrame(index=range(0),columns=df.columns)
transcript_df 

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1


In [19]:


files = os.listdir("Data/scrnaseq")
print(files)
for index,file in enumerate(files):
    df = pd.read_csv("Data/scrnaseq/"+file,index_col=0,delim_whitespace=True)
    df_np = df.to_numpy()
    row_mean = df_np.sum(0)/(df_np != 0).sum(0)
    row_mean = pd.DataFrame(row_mean)
    display(row_mean.transpose())
    nam = file.split('_')[7]
    pd.concat([s1, s2])
    transcript_df = transcript_df.append(row_mean,ignore_index=nam) 
    
   
    

['heathlab_dc_9_17_pbmc_gex_library_112_1.txt', 'heathlab_dc_9_17_pbmc_gex_library_3_1.txt']


C:\Users\jenna\AppData\Local\Temp\ipykernel_17960\235634911.py:6: RuntimeWarning: invalid value encountered in divide
  row_mean = df_np.sum(0)/(df_np != 0).sum(0)


,0,1,2,3,4,5,6,7,8,9,...,24956,24957,24958,24959,24960,24961,24962,24963,24964,24965
0,5.921185,5.83638,5.966526,6.135323,NaN,NaN,NaN,NaN,5.824851,6.162966,...,5.821033,5.625894,6.067718,5.958523,5.922725,6.059433,NaN,6.013412,6.1952,6.086209


C:\Users\jenna\AppData\Local\Temp\ipykernel_17960\235634911.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transcript_df = transcript_df.append(row_mean,ignore_index=nam)
C:\Users\jenna\AppData\Local\Temp\ipykernel_17960\235634911.py:6: RuntimeWarning: invalid value encountered in divide
  row_mean = df_np.sum(0)/(df_np != 0).sum(0)


,0,1,2,3,4,5,6,7,8,9,...,24956,24957,24958,24959,24960,24961,24962,24963,24964,24965
0,5.749782,5.583694,5.346004,5.790983,NaN,NaN,NaN,NaN,5.558157,5.60934,...,5.468164,5.555347,5.487971,5.587403,5.533184,5.549645,5.85979,5.593931,6.124025,5.767535


C:\Users\jenna\AppData\Local\Temp\ipykernel_17960\235634911.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transcript_df = transcript_df.append(row_mean,ignore_index=nam)


KeyboardInterrupt: 

In [ ]:
transcript_df

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1


In [ ]:
#df = sc.read_text("Data/scrnaseq/heathlab_dc_9_17_pbmc_gex_library_3_1.txt", delimiter=None, first_column_names=None, dtype='float32')
df = pd.read_csv("Data/scrnaseq/heathlab_dc_9_17_pbmc_gex_library_3_1.txt",index_col=0,delim_whitespace=True)
df2 = pd.read_csv("Data/scrnaseq/heathlab_dc_9_17_pbmc_gex_library_112_1.txt",index_col=0,delim_whitespace=True)

In [ ]:
#Rows are cells, columns are genes
display(df)
#display(df.iloc[:,:15])
 # AC007952.4	AC245014.3	ACAP1
display(df2)

# 24966 genes 

In [81]:
# 24966 genes ,  961 genes after pseudobulking

df_np = df.to_numpy()
nonzero_counts = np.count_nonzero(df_np,axis=0)
all_zero = []
for i in range(0,len(nonzero_counts)):
    if nonzero_counts[i] == 0:
        all_zero.append(i)

print(len(all_zero))  #7351 are zero


df_np2 = df2.to_numpy()  
nonzero_counts2 = np.count_nonzero(df_np2,axis=0)
all_zero2 = []
for i in range(0,len(nonzero_counts2)):
    if nonzero_counts2[i] == 0:
        all_zero2.append(i)

print(len(all_zero2))  #9110 are zero


len(set(all_zero) & set(all_zero2))  #6335 are both 0

7351
9110


6535

In [ ]:
adata1 = sc.AnnData(df)
adata1.obs["Sample ID"] = 'Sample 3'
adata2 = sc.AnnData(df2)
adata2.obs["Sample ID"] = 'Sample 112'

In [ ]:
#https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html

#see array of gene expression matrix
adata2.X
#see cell names
adata2.obs_names
#see gene names
adata2.var_names
#add annotatation data 
adata2.obs["Sample_ID"] = 'Sample 2'
#check observation annotations
adata2

#Get all cells which are in the 0 and 1-2 class
#adata2[adata.obs.WHO_status.isin(['1-2', '0'])].obs.head()

In [64]:

adata = anndata.concat([adata1, adata2], join='outer',index_unique=None)
adata
#adata[adata.obs.cell_type.isin(['Sample 1'])].obs.head()

AnnData object with n_obs × n_vars = 4566 × 24966
    obs: 'Sample ID'

In [65]:
adata_df = pd.DataFrame(adata.X,columns=adata.var_names,index=adata.obs_names)
adata_df

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
AAACCTGAGTGGGCTA-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTCGAGTTT-1-2:S5:3-1,7.280958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.280958,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTGTCGCTG-1-2:S5:3-1,6.016969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGGTTATCACG-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
AAACCTGTCAAGGTAA-1-2:S5:3-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.287707,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCCTCGTGATAAGT-1-10:J2:112-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,6.167441,0.0
TTTCCTCTCACATACG-1-10:J2:112-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
TTTGGTTGTTGTACAC-1-10:J2:112-1,6.860014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
TTTGGTTTCCTCCTAG-1-10:J2:112-1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,5.851111,0.0


In [66]:

adata_np = adata_df.to_numpy()

nonzero_counts = np.count_nonzero(adata_np,axis=0)

#6536 columns with all zero counts
all_zero = []
for i in range(0,len(nonzero_counts)):
    if nonzero_counts[i] == 0:
        all_zero.append(i)

len(all_zero)

6535

In [ ]:
#see whether the samples are integrated
# Identify highly variable genes
sc.pp.highly_variable_genes(adata, batch_key='Sample ID')

# Scale the data
sc.pp.scale(adata, max_value=10)

# Generate PCA features
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True)

# Compute distances in the PCA space, and find cell neighbors
sc.pp.neighbors(adata)

# Generate UMAP features
sc.tl.umap(adata)


sc.pl.umap(adata, color=['Sample ID'], frameon=False)

In [ ]:
help(dc.get_pseudobulk)

In [84]:
#adata dimensions (4566, 24966)

# Get pseudo-bulk profile
pdata = dc.get_pseudobulk(adata,
                          sample_col = "Sample ID",
                          groups_col = None,
                          use_raw=False,
                          mode = 'mean',
                          min_prop = 0,
                          min_smpls = 0
                         )
pdata

#From documentation: this function removes genes that are not expressed enough across cells (min_prop) or samples (min_smpls)
#Set both to zero so we can retain the genes
#Missing genes are the ones which have expression of zero for all cells

View of AnnData object with n_obs × n_vars = 2 × 18431
    obs: 'Sample ID'

In [85]:
result = pd.DataFrame(pdata.X,columns=pdata.var_names,index=pdata.obs_names)
result

#A1BG	A1BG-AS1	A2M	A2M-AS1	A2ML1	A2ML1-AS1	A3GALT2

,A1BG,A1BG-AS1,A2M,A2M-AS1,AAAS,AACS,AADAT,AAED1,AAGAB,AAK1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
Sample 112,0.623283,0.073550,0.008846,0.031836,0.233167,0.077665,0.003482,0.401872,0.363442,1.133028,...,0.103562,0.050045,0.017992,0.066255,0.079028,0.103311,0.000000,0.156019,2.365106,0.315815
Sample 3,1.077749,0.104141,0.001662,0.043203,0.312722,0.088926,0.000000,0.289758,0.312073,1.678674,...,0.059492,0.048352,0.037530,0.059052,0.049879,0.148359,0.001822,0.215619,3.279979,0.466136


In [ ]:
# Download data-set
import scanpy as sc
adata = sc.datasets.ebi_expression_atlas("E-MTAB-9221", filter_boring=True)


In [ ]:

adata = sc.datasets.ebi_expression_atlas("E-MTAB-9221", filter_boring=True)
adata


In [ ]:
# Rename meta-data
columns = ['Sample Characteristic[sex]',
           'Sample Characteristic[individual]',
           'Sample Characteristic[disease]',
           'Factor Value[inferred cell type - ontology labels]']
adata.obs = adata.obs[columns]
adata.obs.columns = ['sex','individual','disease','cell_type']
adata